In [ ]:
"""
Created on Fri Oct 27 11:29 2023

Plot scatter between temperature anomaly and melt

Author: @claraburgard
"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNxsmall96',
              'NNsmall','NNlarge']

Models

Temperature

In [ ]:
inputpath_tas = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/CMIP_TEMP/'

In [ ]:
mod_list = ['ACCESS-CM2','ACCESS-ESM1-5','CNRM-CM6-1','CNRM-ESM2-1',
            'CESM2-WACCM','CESM2','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0','MPI-ESM1-2-HR',
            'GFDL-ESM4','UKESM1-0-LL'] #'GFDL-CM4','GISS_E2-1-H',

In [ ]:
tas_mod_list = []

for mod in mod_list:
    print(mod)
    if mod not in ['GISS_E2-1-H']:
        tas_scen_list = []
        for scen in ['historical','ssp126','ssp245','ssp585']:
            if (mod == 'GFDL-CM4') and (scen == 'ssp126'):
                tas_mod_scen['tas'] = tas_mod_scen['tas']*np.nan
            else:
                tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_fldmean_ymean.nc')
                tas_mod_scen['time'] = tas_mod_scen.time.dt.year
            tas_out = tas_mod_scen['tas'].squeeze(drop=True)
            if 'height' in tas_out.coords:
                tas_out = tas_out.drop('height')
            tas_scen_list.append(tas_out.assign_coords({'scenario':scen}))

        tas_all_scen = xr.concat(tas_scen_list,dim='scenario')
        tas_mod_list.append(tas_all_scen.assign_coords({'model':mod}))
        
tas_all_mod = xr.concat(tas_mod_list, dim='model')

In [ ]:
anom_mod_scen = tas_all_mod - tas_all_mod.sel(scenario='historical').sel(time=range(1850,1880)).mean('time')

Melt

In [ ]:
## Melt outputpath


Gt_all_scen_list = []
for scenario in ['historical','ssp126','ssp245','ssp585']:

    Gt_allmod_list = []
    box1_allmod_list = []
    for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                'CanESM5', 'GFDL-ESM4','IPSL-CM6A-LR',
               'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']: #'GISS-E2-1-H','GFDL-CM4',

    #for mod in ['ACCESS-ESM1-5']:

        outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

        melt1D_list = []
        for mparam in param_classic_list:
            melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')
            melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
        melt1D_classic = xr.concat(melt1D_list, dim='param')       
        Gt_classic = melt1D_classic['melt_1D_Gt_per_y']

        melt1D_list = []
        for mparam in param_NN_list:
            melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'.nc')
            melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
        melt1D_NN = xr.concat(melt1D_list, dim='param')   
        Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

        Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')

        Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

    Gt_allmod = xr.concat(Gt_allmod_list, dim='model')
    Gt_all_scen_list.append(Gt_allmod.assign_coords({'scenario': scenario}))

Gt_all_scen = xr.concat(Gt_all_scen_list, dim='scenario')
    

Observations

Temperature

In [ ]:
inputpath_raw = home_path+'/DATA/SUMMER_PAPER/raw/'
HadCRUT_temp = xr.open_dataset(inputpath_raw + 'HadCRUT.5.0.0.0.analysis.summary_series.global.annual.nc')
HadCRUT_tas = HadCRUT_temp['tas_mean']
HadCRUT_tas['time'] = HadCRUT_temp.time.dt.year

In [ ]:
anom_baseline = HadCRUT_tas.sel(time=range(1850,1880)).mean('time')
anom_picontrol = HadCRUT_tas - anom_baseline

Melt

In [ ]:
inputpath_Paolo_processed = '/bettik/millanr/DATA_SERVER/ANTARCTICA/OCEANICE/DATASET_FINAL/'
integrated_Paolo = pd.read_csv(inputpath_Paolo_processed + 'Merged_Integrated_melt_rates.csv',index_col=0)
Paolo_Gt_ds = integrated_Paolo.to_xarray()

inputpath_Paolo_processed = '/bettik/millanr/DATA_SERVER/ANTARCTICA/OCEANICE/DATASET_FINAL/'
integrated_Paolo_unc = pd.read_csv(inputpath_Paolo_processed + 'Merged_Integrated_melt_err_rates.csv',index_col=0)
Paolo_Gt_unc_ds = integrated_Paolo_unc.to_xarray()

In [ ]:
year_list = ['1992'] + np.arange(1994,2015).astype(str).tolist()
merged_list = []
merged_list_unc = []
for vvar in year_list:
    merged_list.append(Paolo_Gt_ds[vvar].rename('melt_Gt').assign_coords({'time': int(vvar)}))
    merged_list_unc.append(Paolo_Gt_unc_ds[vvar].rename('melt_Gt_unc').assign_coords({'time': int(vvar)}))
merged_ds = xr.concat(merged_list, dim='time')
merged_unc_ds = xr.concat(merged_list_unc, dim='time')
merged_ds_both = xr.merge([merged_ds,merged_unc_ds])

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
melt_Gt_list = []
for kisf in file_isf.Nisf:
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    if nname in merged_ds['Names']:
        melt_Gt_kisf = merged_ds_both.sel(Names=nname).squeeze().drop('Names')
    elif nname == 'Conger/Glenzer':
        melt_Gt_kisf = merged_ds_both.sel(Names='Conger_Glenzer').squeeze().drop('Names')
    elif nname == 'George VI':
        melt_Gt_kisf = merged_ds_both.sel(Names='George_VI').squeeze().drop('Names')
    elif str(nname)[0:7] == 'Larsen ':
        melt_Gt_kisf = merged_ds_both.sel(Names='Larsen'+str(nname)[7]).squeeze().drop('Names')
    elif nname == 'Prince Harald':
        melt_Gt_kisf = merged_ds_both.sel(Names='Prince_Harald').squeeze().drop('Names')
    elif nname == 'Moscow Univ.':    
        melt_Gt_kisf = merged_ds_both.sel(Names='Moscow_University').squeeze().drop('Names')
    elif nname == 'Roi Baudouin':
        melt_Gt_kisf = merged_ds_both.sel(Names='Baudouin').squeeze().drop('Names')
    elif nname == 'Wilma/Robert/Downer':
        melt_Gt_kisf = merged_ds_both.sel(Names='WilmaRobertDowner').squeeze().drop('Names')
    elif nname == 'Tracy Tremenchus':
        melt_Gt_kisf = merged_ds_both.sel(Names='Tracy_Tremenchus').squeeze().drop('Names')
    elif nname == 'Rayner/Thyer':
        melt_Gt_kisf = merged_ds_both.sel(Names='Rayner_Thyer').squeeze().drop('Names')
    elif nname == 'Pine Island':
        melt_Gt_kisf = merged_ds_both.sel(Names='Pine_Island').squeeze().drop('Names')
    elif nname == 'Stancomb Brunt':
        melt_Gt_kisf = merged_ds_both.sel(Names='Brunt_Stancomb').squeeze().drop('Names')
    elif nname == 'Edward VIII':
        melt_Gt_kisf = merged_ds_both.sel(Names='Edward_VIII').squeeze().drop('Names')
    elif nname == 'Vincennes Bay':
        melt_Gt_kisf = merged_ds_both.sel(Names='Vincennes_Bay').squeeze().drop('Names')
    elif nname == 'Filchner-Ronne':
        melt_Gt_kisf = merged_ds_both.sel(Names='Ronne').squeeze().drop('Names') + merged_ds_both.sel(Names='Filchner').squeeze().drop('Names')
    elif nname == 'Ross':
        melt_Gt_kisf = merged_ds_both.sel(Names='Ross_East').squeeze().drop('Names') + merged_ds_both.sel(Names='Ross_West').squeeze().drop('Names')
    elif nname == 'Wordie':
        melt_Gt_kisf = merged_ds_both.sel(Names='Wordie_(Harriott)').squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Harriott_Headland)'
                                                                               ).squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Cape_Jeremy)'
                                                                               ).squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Prospect)'
                                                                               ).squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Airy_Rotz_Seller)').squeeze().drop('Names')
    melt_Gt_list.append(melt_Gt_kisf.assign_coords({'Nisf': kisf}))

melt_Gt_all_Paolo = xr.concat(melt_Gt_list, dim='Nisf')
        


In [ ]:
temp_range_obs = [anom_picontrol.sel(time=melt_Gt_all_Paolo.time).min().values,anom_picontrol.sel(time=melt_Gt_all_Paolo.time).max().values]

In [ ]:
temp_range_obs

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = melt_Gt_all_Paolo['melt_Gt'].sel(Nisf=kisf).values

    ax[i].scatter(anom_picontrol.sel(time=melt_Gt_all_Paolo.time),melt_Gt_all_Paolo['melt_Gt'].sel(Nisf=kisf))



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()


find melt for same temperature anomaly range

In [ ]:
time_mod = anom_mod_scen.time.where(((anom_mod_scen >= temp_range_obs[0]) & (anom_mod_scen <= temp_range_obs[1]))).sel(scenario='historical').dropna('time',how='all')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = melt_Gt_all_Paolo['melt_Gt'].sel(Nisf=kisf).values

    ax[i].scatter(anom_picontrol.sel(time=melt_Gt_all_Paolo.time),melt_Gt_all_Paolo['melt_Gt'].sel(Nisf=kisf))
    for mod in Gt_all_scen.model:
        time_mod0 = time_mod.sel(model=mod).dropna('time')
        time_mod0 = time_mod0.where(time_mod0>=1980,drop=True)
        for mparam in Gt_all_scen.param:
            ax[i].scatter(anom_mod_scen.sel(model=mod,time=time_mod0,scenario='historical'),Gt_all_scen.sel(time=time_mod0,Nisf=kisf,scenario='historical',model=mod,param=mparam), color='k',s=10,alpha=0.1)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'scatter_temp_melt_historical.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    for mod in Gt_all_scen.model:
        for scen in anom_mod_scen.scenario:
            if scen == 'historical':
                ccolor='grey'
            elif scen == 'ssp126':
                ccolor='orange'
            elif scen == 'ssp245':
                ccolor='maroon' 
            elif scen == 'ssp585':
                ccolor='red'   
            for mparam in Gt_all_scen.param:
                ax[i].scatter(anom_mod_scen.sel(time=range(1980,2301),scenario=scen,model=mod),Gt_all_scen.sel(time=range(1980,2301),Nisf=kisf,scenario=scen,model=mod,param=mparam),
                          alpha=0.1,s=10,color=ccolor)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'scatter_temp_melt_allscenarios.png',dpi=300)

In [ ]:
plt.figure()
for scen in anom_mod_scen.scenario:
    for mparam in Gt_all_scen.param:
        plt.scatter(anom_mod_scen.sel(time=range(1980,2101),scenario=scen),Gt_all_scen.sum('Nisf').sel(time=range(1980,2101),scenario=scen,model=mod,param=mparam),
                      alpha=0.1,s=10)